In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
def timestamp_column(df,col):
    df['year'] = df[col].dt.year
    df['month'] = df[col].dt.month
    df['day'] = df[col].dt.day
    df.rename(columns={'year':col+'_year','month':col+'_month','day':col+'_day'},inplace=True)
    df.drop(col,axis=1,inplace=True)
    return df

In [3]:
'''
pid-Unique identifier for each payment.
pdate-Date of payment
pamt-Amount of payment
Extrelno-Unique identifier of each donor
Paytypecd- Paytype
O Bank transfer
D Permanent order
E Own initiative
X Unkonown

Status
Status of payment
OK   Normal/Real payment
CO   Correction (internal)
RF    RF (Refund)
RC    Recall
'''
payhistory=pd.read_sas("payhistory.sas7bdat")

payhistory.iloc[:,3]=payhistory.iloc[:,3].str.decode("utf-8") 
payhistory.iloc[:,4]=payhistory.iloc[:,4].str.decode("utf-8")
payhistory.iloc[:,5]=payhistory.iloc[:,5].str.decode("utf-8")

payhistory.head()

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
0,38.0,1993-12-31,9.57,0000019,X,CO
1,39.0,1993-12-31,41.32,0000020,X,CO
2,40.0,1993-12-31,13.10,0000020,X,CO
3,54091.0,1994-01-01,2.02,0000020,X,CO
4,104480.0,1994-05-19,0.50,0000020,D,OK


In [4]:
# Sorting the payhistory by EXTRELNO .

payhistory.sort_values(['EXTRELNO'],ascending=True,inplace=True)

In [5]:
payhistory.head()

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
0,38.0,1993-12-31,9.57,0000019,X,CO
102,3309171.0,2002-05-14,20.33,0000020,D,OK
103,3345085.0,2002-06-11,20.33,0000020,D,OK
104,3382129.0,2002-07-11,23.00,0000020,D,OK
105,3418584.0,2002-08-12,23.00,0000020,D,OK


In [6]:
#PID have too many unique values,and hence not important 
payhistory.drop(['PID'],axis=1,inplace=True)

In [7]:
pd.DataFrame({'Total':payhistory.isnull().sum(),'Percentage':payhistory.isnull().sum()/payhistory.shape[0]*100})

,Total,Percentage
PDATE,0,0.000000
PAMT,0,0.000000
EXTRELNO,0,0.000000
PAYTYPECD,18174,1.507315
STATUS,1,0.000083


In [8]:
payhistory.PAYTYPECD.value_counts()

D    867875
O    253729
E     46389
X     19553
Name: PAYTYPECD, dtype: int64

In [9]:
payhistory.PAYTYPECD.describe()

count     1187546
unique          4
top             D
freq       867875
Name: PAYTYPECD, dtype: object

In [10]:
#Imputing D(Permanent Order) for the missing values in the PAYTYPECD column
payhistory['PAYTYPECD'].fillna('D',inplace=True)

In [11]:
payhistory['STATUS'].describe()

count     1205719
unique          4
top            OK
freq      1149504
Name: STATUS, dtype: object

In [12]:
# Imputing missing value in STATUS column by the most frequent used category
payhistory['STATUS'].fillna("S",inplace=True)

In [13]:
payhistory.isnull().sum()

PDATE        0
PAMT         0
EXTRELNO     0
PAYTYPECD    0
STATUS       0
dtype: int64

In [14]:
# Hence all the missing values were Imputed/Handled

In [15]:
frequency = payhistory.EXTRELNO.value_counts().to_frame()
frequency.rename(columns={'EXTRELNO':'Frequency'},inplace=True)

frequency['EXTRELNO'] = frequency.index

frequency.reset_index(inplace=True,drop=True)


In [16]:
payhistory1 = pd.merge(payhistory,frequency,on=['EXTRELNO'])
payhistory1.head()

,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS,Frequency
0,1993-12-31,9.57,0000019,X,CO,1
1,2002-05-14,20.33,0000020,D,OK,158
2,2002-06-11,20.33,0000020,D,OK,158
3,2002-07-11,23.00,0000020,D,OK,158
4,2002-08-12,23.00,0000020,D,OK,158


In [17]:
total = payhistory[['EXTRELNO','PAMT']].groupby(['EXTRELNO']).sum() 
total.rename(columns={'PAMT':'TOTAL_DONATION'},inplace=True)

total['EXTRELNO'] = total.index

total.reset_index(drop=True,inplace=True)
total.head()

,TOTAL_DONATION,EXTRELNO
0,9.57,0000019
1,3176.97,0000020
2,3168.40,0000035
3,1543.03,0000042
4,4373.24,0000043


In [18]:
payhistory1 = pd.merge(payhistory1,total,on=['EXTRELNO'])
payhistory1.head()

,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS,Frequency,TOTAL_DONATION
0,1993-12-31,9.57,0000019,X,CO,1,9.57
1,2002-05-14,20.33,0000020,D,OK,158,3176.97
2,2002-06-11,20.33,0000020,D,OK,158,3176.97
3,2002-07-11,23.00,0000020,D,OK,158,3176.97
4,2002-08-12,23.00,0000020,D,OK,158,3176.97


In [19]:
payhistory1.drop(['PAMT'],axis=1,inplace=True)

In [20]:
payhistory1['Average_donation'] = payhistory1['TOTAL_DONATION']/payhistory1['Frequency']
payhistory1.head()

,PDATE,EXTRELNO,PAYTYPECD,STATUS,Frequency,TOTAL_DONATION,Average_donation
0,1993-12-31,0000019,X,CO,1,9.57,9.570000
1,2002-05-14,0000020,D,OK,158,3176.97,20.107405
2,2002-06-11,0000020,D,OK,158,3176.97,20.107405
3,2002-07-11,0000020,D,OK,158,3176.97,20.107405
4,2002-08-12,0000020,D,OK,158,3176.97,20.107405


In [21]:
payhistory1.PAYTYPECD.value_counts()

D    886049
O    253729
E     46389
X     19553
Name: PAYTYPECD, dtype: int64

In [22]:
a = pd.get_dummies(payhistory1['PAYTYPECD'],prefix='PAYTYPECD')
payhistory1 = payhistory1.join(a)
payhistory1.drop(['PAYTYPECD'],axis=1,inplace=True)


In [23]:
payhistory1.head()

,PDATE,EXTRELNO,STATUS,Frequency,TOTAL_DONATION,Average_donation,PAYTYPECD_D,PAYTYPECD_E,PAYTYPECD_O,PAYTYPECD_X
0,1993-12-31,0000019,CO,1,9.57,9.570000,0,0,0,1
1,2002-05-14,0000020,OK,158,3176.97,20.107405,1,0,0,0
2,2002-06-11,0000020,OK,158,3176.97,20.107405,1,0,0,0
3,2002-07-11,0000020,OK,158,3176.97,20.107405,1,0,0,0
4,2002-08-12,0000020,OK,158,3176.97,20.107405,1,0,0,0


In [24]:
payhistory1.sort_values(by=['EXTRELNO', 'PDATE'], ascending=[True,False],inplace=True) 

payhistory1.head()

,PDATE,EXTRELNO,STATUS,Frequency,TOTAL_DONATION,Average_donation,PAYTYPECD_D,PAYTYPECD_E,PAYTYPECD_O,PAYTYPECD_X
0,1993-12-31,0000019,CO,1,9.57,9.570000,0,0,0,1
56,2007-01-17,0000020,OK,158,3176.97,20.107405,1,0,0,0
55,2006-12-08,0000020,OK,158,3176.97,20.107405,1,0,0,0
54,2006-11-10,0000020,OK,158,3176.97,20.107405,1,0,0,0
53,2006-10-06,0000020,OK,158,3176.97,20.107405,1,0,0,0


In [25]:
# This would be added to the unique payhistory column
total_paytype = payhistory1[['EXTRELNO','PAYTYPECD_D','PAYTYPECD_E','PAYTYPECD_O','PAYTYPECD_X']].groupby(['EXTRELNO']).sum()
# i am replacing every value greater than zero to be 1 and else 0
total_paytype = total_paytype.applymap(lambda x: 1 if x>=1 else 0)
total_paytype.head()

,PAYTYPECD_D,PAYTYPECD_E,PAYTYPECD_O,PAYTYPECD_X
EXTRELNO,,,,
0000019,0,0,0,1
0000020,1,0,0,1
0000035,1,0,0,1
0000042,1,0,0,1
0000043,1,0,0,1


In [26]:
# Also dropping the paytypecd dummies from the payhistory1

payhistory1.drop(a,axis=1,inplace=True)
payhistory1.head()

,PDATE,EXTRELNO,STATUS,Frequency,TOTAL_DONATION,Average_donation
0,1993-12-31,0000019,CO,1,9.57,9.570000
56,2007-01-17,0000020,OK,158,3176.97,20.107405
55,2006-12-08,0000020,OK,158,3176.97,20.107405
54,2006-11-10,0000020,OK,158,3176.97,20.107405
53,2006-10-06,0000020,OK,158,3176.97,20.107405


In [27]:
payhistory1.drop_duplicates(['EXTRELNO'],inplace=True)
payhistory1 = pd.merge(payhistory1,total_paytype,on='EXTRELNO')

In [28]:
payhistory1.head()

,PDATE,EXTRELNO,STATUS,Frequency,TOTAL_DONATION,Average_donation,PAYTYPECD_D,PAYTYPECD_E,PAYTYPECD_O,PAYTYPECD_X
0,1993-12-31,0000019,CO,1,9.57,9.570000,0,0,0,1
1,2007-01-17,0000020,OK,158,3176.97,20.107405,1,0,0,1
2,2007-01-17,0000035,OK,155,3168.40,20.441290,1,0,0,1
3,2001-01-23,0000042,RC,85,1543.03,18.153294,1,0,0,1
4,2003-07-14,0000043,OK,229,4373.24,19.097118,1,0,0,1


In [29]:
# Getting dummies for STATUS column too 
payhistory1 = pd.get_dummies(payhistory1,columns=['STATUS'])

In [30]:
payhistory1.index = payhistory1['EXTRELNO']
payhistory1.drop(['EXTRELNO'],axis=1,inplace=True)
payhistory1.head()

,PDATE,Frequency,TOTAL_DONATION,Average_donation,PAYTYPECD_D,PAYTYPECD_E,PAYTYPECD_O,PAYTYPECD_X,STATUS_CO,STATUS_OK,STATUS_RC,STATUS_RF
EXTRELNO,,,,,,,,,,,,
0000019,1993-12-31,1,9.57,9.570000,0,0,0,1,1,0,0,0
0000020,2007-01-17,158,3176.97,20.107405,1,0,0,1,0,1,0,0
0000035,2007-01-17,155,3168.40,20.441290,1,0,0,1,0,1,0,0
0000042,2001-01-23,85,1543.03,18.153294,1,0,0,1,0,0,1,0
0000043,2003-07-14,229,4373.24,19.097118,1,0,0,1,0,1,0,0


In [31]:
payhistory1 = timestamp_column(payhistory1,'PDATE')
payhistory1.head()

,Frequency,TOTAL_DONATION,Average_donation,PAYTYPECD_D,PAYTYPECD_E,PAYTYPECD_O,PAYTYPECD_X,STATUS_CO,STATUS_OK,STATUS_RC,STATUS_RF,PDATE_year,PDATE_month,PDATE_day
EXTRELNO,,,,,,,,,,,,,,
0000019,1,9.57,9.570000,0,0,0,1,1,0,0,0,1993,12,31
0000020,158,3176.97,20.107405,1,0,0,1,0,1,0,0,2007,1,17
0000035,155,3168.40,20.441290,1,0,0,1,0,1,0,0,2007,1,17
0000042,85,1543.03,18.153294,1,0,0,1,0,0,1,0,2001,1,23
0000043,229,4373.24,19.097118,1,0,0,1,0,1,0,0,2003,7,14


In [32]:
'''
Communication:    All    possible communication    between    the    donor    and    the 
organization
Contid-Unique identifier for each contact
Mediumcode-Medium of the contact (CI is unknown)
Mntopcode-Main topic code of the contact
Classcode-Class of the contact
Extrelno-Unique identifier for each donor
Contdirec-Direction of the communication
I-Incoming
P-Outgoing
Contdate-Date of the contact

'''
communication=pd.read_sas("communication.sas7bdat")

for col in communication.columns:
    if col != 'CONTDATE':
        communication[col]=communication[col].str.decode("utf-8")
        
communication.head()

,CONTID,MEDIUMCODE,MNTOPCODE,CLASCODE,EXTRELNO,CONTDIREC,CONTDATE
0,000405402,CI,CTS,CT,0000020,O,1994-05-09
1,000430904,CI,CTS,CT,0000020,O,1994-06-14
2,000592732,CI,CTS,CT,0000020,O,1994-10-21
3,000593375,CI,CTS,CT,0000020,O,1994-10-21
4,000618514,CI,CTS,CT,0000020,O,1994-11-28


In [33]:
'''
Comclas-Description of the contact class.
Clascode- Code of contact class
Clasdesc- Description

'''
comclas=pd.read_sas("comclas.sas7bdat")

for col in comclas.columns:
    comclas[col]=comclas[col].str.decode("utf-8")
comclas.head()

,CLASCODE,CLASDESC
0,CT,CTS-item
1,RI,Request for information
2,CM,Complaint
3,TA,Threath / Accusation
4,AK,Acknowledgement


In [34]:
'''
Commaint-Description of the main topic code.
Mntopcode- Main topic code
Mntopdesc- Description

'''
commaint=pd.read_sas("commaint.sas7bdat")

for col in commaint.columns:
    commaint[col]=commaint[col].str.decode("utf-8")
commaint.head()

,MNTOPCODE,MNTOPDESC
0,CAN,Cancellations/replacements
1,DIS,Discontinuations
2,MKT,Marketing - Enquiries
3,GEN,General
4,NEW,New relations


In [35]:
'''
Commediu: Description of medium type.
Mediumcode- Code of the mediumtype
Mediumdesc- Description

'''
commediu=pd.read_sas("commediu.sas7bdat")

for col in commediu.columns:
    commediu[col]=commediu[col].str.decode("utf-8")
commediu.head()

,MEDIUMCODE,MEDIUMDESC
0,FF,Face to Face contact
1,FX,Fax
2,TX,Telex
3,EM,Electronic Mail
4,TE,Telephone call


In [36]:
#Sorting by EXTRELNO
communication.sort_values(['EXTRELNO'],ascending=True,inplace=True)

In [37]:
#There are no null values 
communication.isnull().sum()

CONTID        0
MEDIUMCODE    0
MNTOPCODE     0
CLASCODE      0
EXTRELNO      0
CONTDIREC     0
CONTDATE      0
dtype: int64

In [38]:
#Dropping the CONTID column
communication.drop(['CONTID'],axis=1,inplace=True)

In [39]:
communication = pd.get_dummies(data=communication,columns=['MEDIUMCODE','MNTOPCODE','CLASCODE','CONTDIREC'])
communication.head()

,EXTRELNO,CONTDATE,MEDIUMCODE_AC,MEDIUMCODE_AF,MEDIUMCODE_AM,MEDIUMCODE_CA,MEDIUMCODE_CI,MEDIUMCODE_EM,MEDIUMCODE_FF,MEDIUMCODE_FX,...,CLASCODE_AN,CLASCODE_CM,CLASCODE_CT,CLASCODE_ID,CLASCODE_NS,CLASCODE_PM,CLASCODE_RI,CLASCODE_RM,CONTDIREC_I,CONTDIREC_O
0,0000020,1994-05-09,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
29,0000020,2001-02-27,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
30,0000020,2001-03-12,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
31,0000020,2001-10-29,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
32,0000020,2001-11-20,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [40]:
communication.columns

Index(['EXTRELNO', 'CONTDATE', 'MEDIUMCODE_AC', 'MEDIUMCODE_AF',
       'MEDIUMCODE_AM', 'MEDIUMCODE_CA', 'MEDIUMCODE_CI', 'MEDIUMCODE_EM',
       'MEDIUMCODE_FF', 'MEDIUMCODE_FX', 'MEDIUMCODE_LE', 'MEDIUMCODE_ME',
       'MEDIUMCODE_MR', 'MEDIUMCODE_NA', 'MEDIUMCODE_RM', 'MEDIUMCODE_TE',
       'MEDIUMCODE_TM', 'MNTOPCODE_ALL', 'MNTOPCODE_ARR', 'MNTOPCODE_CAN',
       'MNTOPCODE_CAT', 'MNTOPCODE_COR', 'MNTOPCODE_CTS', 'MNTOPCODE_DIS',
       'MNTOPCODE_ENR', 'MNTOPCODE_FIC', 'MNTOPCODE_FIN', 'MNTOPCODE_IFF',
       'MNTOPCODE_INF', 'MNTOPCODE_IRE', 'MNTOPCODE_MER', 'MNTOPCODE_PRA',
       'MNTOPCODE_PRO', 'MNTOPCODE_REP', 'MNTOPCODE_SPO', 'MNTOPCODE_TAX',
       'MNTOPCODE_TLT', 'MNTOPCODE_TVS', 'MNTOPCODE_VIS', 'MNTOPCODE_VOL',
       'MNTOPCODE_WEB', 'CLASCODE_AK', 'CLASCODE_AN', 'CLASCODE_CM',
       'CLASCODE_CT', 'CLASCODE_ID', 'CLASCODE_NS', 'CLASCODE_PM',
       'CLASCODE_RI', 'CLASCODE_RM', 'CONTDIREC_I', 'CONTDIREC_O'],
      dtype='object')

In [41]:
communication_finaladd = communication[['EXTRELNO','MEDIUMCODE_AC', 'MEDIUMCODE_AF',
       'MEDIUMCODE_AM', 'MEDIUMCODE_CA', 'MEDIUMCODE_CI', 'MEDIUMCODE_EM',
       'MEDIUMCODE_FF', 'MEDIUMCODE_FX', 'MEDIUMCODE_LE', 'MEDIUMCODE_ME',
       'MEDIUMCODE_MR', 'MEDIUMCODE_NA', 'MEDIUMCODE_RM', 'MEDIUMCODE_TE',
       'MEDIUMCODE_TM', 'MNTOPCODE_ALL', 'MNTOPCODE_ARR', 'MNTOPCODE_CAN',
       'MNTOPCODE_CAT', 'MNTOPCODE_COR', 'MNTOPCODE_CTS', 'MNTOPCODE_DIS',
       'MNTOPCODE_ENR', 'MNTOPCODE_FIC', 'MNTOPCODE_FIN', 'MNTOPCODE_IFF',
       'MNTOPCODE_INF', 'MNTOPCODE_IRE', 'MNTOPCODE_MER', 'MNTOPCODE_PRA',
       'MNTOPCODE_PRO', 'MNTOPCODE_REP', 'MNTOPCODE_SPO', 'MNTOPCODE_TAX',
       'MNTOPCODE_TLT', 'MNTOPCODE_TVS', 'MNTOPCODE_VIS', 'MNTOPCODE_VOL',
       'MNTOPCODE_WEB', 'CLASCODE_AK', 'CLASCODE_AN', 'CLASCODE_CM',
       'CLASCODE_CT', 'CLASCODE_ID', 'CLASCODE_NS', 'CLASCODE_PM',
       'CLASCODE_RI', 'CLASCODE_RM', 'CONTDIREC_I', 'CONTDIREC_O']].groupby(['EXTRELNO']).sum()
communication_finaladd = communication_finaladd.applymap(lambda x: 1 if x>=1 else 0)
communication_finaladd.head()

,MEDIUMCODE_AC,MEDIUMCODE_AF,MEDIUMCODE_AM,MEDIUMCODE_CA,MEDIUMCODE_CI,MEDIUMCODE_EM,MEDIUMCODE_FF,MEDIUMCODE_FX,MEDIUMCODE_LE,MEDIUMCODE_ME,...,CLASCODE_AN,CLASCODE_CM,CLASCODE_CT,CLASCODE_ID,CLASCODE_NS,CLASCODE_PM,CLASCODE_RI,CLASCODE_RM,CONTDIREC_I,CONTDIREC_O
EXTRELNO,,,,,,,,,,,,,,,,,,,,,
0000020,0,0,0,0,1,1,0,0,1,0,...,1,0,1,1,0,1,0,0,0,1
0000035,0,0,0,0,1,1,0,0,1,0,...,1,0,1,1,0,1,0,0,1,1
0000042,0,0,0,0,1,0,0,0,1,0,...,1,0,1,1,0,0,1,1,1,1
0000043,0,0,0,0,1,1,0,0,1,0,...,1,0,1,1,0,1,1,0,1,1
0000060,0,0,0,0,1,0,0,0,1,0,...,1,0,1,1,0,0,1,0,1,1


In [42]:
communication.drop(['MEDIUMCODE_AC', 'MEDIUMCODE_AF',
       'MEDIUMCODE_AM', 'MEDIUMCODE_CA', 'MEDIUMCODE_CI', 'MEDIUMCODE_EM',
       'MEDIUMCODE_FF', 'MEDIUMCODE_FX', 'MEDIUMCODE_LE', 'MEDIUMCODE_ME',
       'MEDIUMCODE_MR', 'MEDIUMCODE_NA', 'MEDIUMCODE_RM', 'MEDIUMCODE_TE',
       'MEDIUMCODE_TM', 'MNTOPCODE_ALL', 'MNTOPCODE_ARR', 'MNTOPCODE_CAN',
       'MNTOPCODE_CAT', 'MNTOPCODE_COR', 'MNTOPCODE_CTS', 'MNTOPCODE_DIS',
       'MNTOPCODE_ENR', 'MNTOPCODE_FIC', 'MNTOPCODE_FIN', 'MNTOPCODE_IFF',
       'MNTOPCODE_INF', 'MNTOPCODE_IRE', 'MNTOPCODE_MER', 'MNTOPCODE_PRA',
       'MNTOPCODE_PRO', 'MNTOPCODE_REP', 'MNTOPCODE_SPO', 'MNTOPCODE_TAX',
       'MNTOPCODE_TLT', 'MNTOPCODE_TVS', 'MNTOPCODE_VIS', 'MNTOPCODE_VOL',
       'MNTOPCODE_WEB', 'CLASCODE_AK', 'CLASCODE_AN', 'CLASCODE_CM',
       'CLASCODE_CT', 'CLASCODE_ID', 'CLASCODE_NS', 'CLASCODE_PM',
       'CLASCODE_RI', 'CLASCODE_RM', 'CONTDIREC_I', 'CONTDIREC_O'],axis=1,inplace=True)

In [43]:
communication.drop_duplicates(['EXTRELNO'],inplace=True)

In [44]:
communication = pd.merge(communication,communication_finaladd,on='EXTRELNO')
communication.sort_values(['EXTRELNO'],inplace=True)


In [45]:
communication.reset_index(inplace=True)
communication.drop(['index'],axis=1,inplace=True)
communication

,EXTRELNO,CONTDATE,MEDIUMCODE_AC,MEDIUMCODE_AF,MEDIUMCODE_AM,MEDIUMCODE_CA,MEDIUMCODE_CI,MEDIUMCODE_EM,MEDIUMCODE_FF,MEDIUMCODE_FX,...,CLASCODE_AN,CLASCODE_CM,CLASCODE_CT,CLASCODE_ID,CLASCODE_NS,CLASCODE_PM,CLASCODE_RI,CLASCODE_RM,CONTDIREC_I,CONTDIREC_O
0,0000020,1994-05-09,0,0,0,0,1,1,0,0,...,1,0,1,1,0,1,0,0,0,1
1,0000035,2002-03-21,0,0,0,0,1,1,0,0,...,1,0,1,1,0,1,0,0,1,1
2,0000042,1998-12-03,0,0,0,0,1,0,0,0,...,1,0,1,1,0,0,1,1,1,1
3,0000043,2000-05-05,0,0,0,0,1,1,0,0,...,1,0,1,1,0,1,1,0,1,1
4,0000060,1998-03-20,0,0,0,0,1,0,0,0,...,1,0,1,1,0,0,1,0,1,1
5,0000072,2002-03-21,0,0,0,0,1,1,0,0,...,1,0,1,1,0,1,1,0,1,1
6,0000081,2001-10-24,0,0,0,0,1,1,0,1,...,1,1,1,1,0,1,1,0,1,1
7,0000089,2002-11-07,0,0,0,0,1,1,0,0,...,1,0,1,1,0,1,0,0,0,1
8,0000091,2002-02-16,0,0,0,0,1,1,0,0,...,1,0,1,1,0,1,1,0,1,1
9,0000100,2002-03-21,0,0,0,0,1,1,0,0,...,1,0,1,1,0,1,0,0,0,1


In [46]:
communication.index=communication['EXTRELNO']
communication.drop(['EXTRELNO'],axis=1,inplace=True)
communication.head()

,CONTDATE,MEDIUMCODE_AC,MEDIUMCODE_AF,MEDIUMCODE_AM,MEDIUMCODE_CA,MEDIUMCODE_CI,MEDIUMCODE_EM,MEDIUMCODE_FF,MEDIUMCODE_FX,MEDIUMCODE_LE,...,CLASCODE_AN,CLASCODE_CM,CLASCODE_CT,CLASCODE_ID,CLASCODE_NS,CLASCODE_PM,CLASCODE_RI,CLASCODE_RM,CONTDIREC_I,CONTDIREC_O
EXTRELNO,,,,,,,,,,,,,,,,,,,,,
0000020,1994-05-09,0,0,0,0,1,1,0,0,1,...,1,0,1,1,0,1,0,0,0,1
0000035,2002-03-21,0,0,0,0,1,1,0,0,1,...,1,0,1,1,0,1,0,0,1,1
0000042,1998-12-03,0,0,0,0,1,0,0,0,1,...,1,0,1,1,0,0,1,1,1,1
0000043,2000-05-05,0,0,0,0,1,1,0,0,1,...,1,0,1,1,0,1,1,0,1,1
0000060,1998-03-20,0,0,0,0,1,0,0,0,1,...,1,0,1,1,0,0,1,0,1,1


In [47]:
communication = timestamp_column(communication,'CONTDATE')
communication.head()

,MEDIUMCODE_AC,MEDIUMCODE_AF,MEDIUMCODE_AM,MEDIUMCODE_CA,MEDIUMCODE_CI,MEDIUMCODE_EM,MEDIUMCODE_FF,MEDIUMCODE_FX,MEDIUMCODE_LE,MEDIUMCODE_ME,...,CLASCODE_ID,CLASCODE_NS,CLASCODE_PM,CLASCODE_RI,CLASCODE_RM,CONTDIREC_I,CONTDIREC_O,CONTDATE_year,CONTDATE_month,CONTDATE_day
EXTRELNO,,,,,,,,,,,,,,,,,,,,,
0000020,0,0,0,0,1,1,0,0,1,0,...,1,0,1,0,0,0,1,1994,5,9
0000035,0,0,0,0,1,1,0,0,1,0,...,1,0,1,0,0,1,1,2002,3,21
0000042,0,0,0,0,1,0,0,0,1,0,...,1,0,0,1,1,1,1,1998,12,3
0000043,0,0,0,0,1,1,0,0,1,0,...,1,0,1,1,0,1,1,2000,5,5
0000060,0,0,0,0,1,0,0,0,1,0,...,1,0,0,1,0,1,1,1998,3,20


In [48]:
'''
Extrel: All the donors of the organization.
Extrelno-Unique identifier of each donor
Exrelactcd-Activity code of the donor
Extrelstdt-Start date of the relationship
Exreldaten-End date of the relationship (Missing: not ended)
'''
extrel=pd.read_sas("extrel.sas7bdat")

extrel['EXTRELNO']=extrel.iloc[:,0].str.decode("utf-8")
extrel['EXRELACTCD']=extrel.iloc[:,1].str.decode("utf-8")

extrel.head()

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
0,0026414,CT,1995-02-02,NaT
1,0026419,FP,1992-03-01,NaT
2,0026424,FP,1992-03-01,1996-12-24
3,0026430,FP,1992-03-01,2006-01-31
4,0026430,CT,1997-03-07,NaT


In [49]:
'''
Extrelty: Description of the activity
Exrelactcd-Activity code of the donor
Exrelactde-Description of the activity
'''
extrelty=pd.read_sas("extrelty.sas7bdat")

for col in extrelty.columns:
    extrelty[col]=extrelty[col].str.decode("utf-8")
    
extrelty.head()

,EXRELACTCD,EXRELACTDE
0,FP,CommitmentDo
1,PS,Prospect
2,PF,Prosp Fundra
3,PP,Prosp Promot
4,GR,Grantor


In [50]:
# AS we only need to select donors with commitment, So i would be slecting only donore with FP EXRELACTCD.
extrel = extrel[extrel['EXRELACTCD']=='FP']
extrel.head()

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
1,0026419,FP,1992-03-01,NaT
2,0026424,FP,1992-03-01,1996-12-24
3,0026430,FP,1992-03-01,2006-01-31
5,0026431,FP,1992-03-01,NaT
7,0026443,FP,1992-03-01,NaT


In [51]:

extrel.sort_values(by=['EXTRELNO', 'EXRELDATEN'], ascending=[True,True],inplace=True)

In [52]:
#Checking for all the missing values in the dataframe
extrel.isnull().sum()

EXTRELNO         0
EXRELACTCD       0
EXTRELSTDT       0
EXRELDATEN    7075
dtype: int64

In [53]:
extrel.head(10)

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
1,0026419,FP,1992-03-01,NaT
2,0026424,FP,1992-03-01,1996-12-24
3,0026430,FP,1992-03-01,2006-01-31
5,0026431,FP,1992-03-01,NaT
7,0026443,FP,1992-03-01,NaT
8,0026444,FP,1992-03-01,NaT
9,0026446,FP,1992-03-01,NaT
10,0026447,FP,1992-03-01,NaT
12,0026462,FP,1992-03-01,2001-10-18
13,0026467,FP,1992-03-01,2005-01-06


In [54]:
# FOR all the missing in EXTRELDATEN , i am imputing 0 as they have not ended(not churned)
extrel.EXRELDATEN.fillna(0,inplace=True)

# Creating a column named Churned that signifies (1 : Churned and 0 : Not Churned)
extrel['Churned']=0
extrel.loc[extrel['EXRELDATEN']==0,'Churned']=1

#Dropping the exreldaten column 
extrel.drop(['EXRELDATEN'],axis=1,inplace=True)
extrel.head()

,EXTRELNO,EXRELACTCD,EXTRELSTDT,Churned
1,0026419,FP,1992-03-01,1
2,0026424,FP,1992-03-01,0
3,0026430,FP,1992-03-01,0
5,0026431,FP,1992-03-01,1
7,0026443,FP,1992-03-01,1


In [55]:
extrel.drop_duplicates(['EXTRELNO'],inplace=True)

extrel.shape

(11242, 4)

In [56]:
#Dropping EXRELACTCD column as it is having same type of values
extrel.drop(['EXRELACTCD'],axis=1,inplace=True)

In [57]:
extrel.index = extrel['EXTRELNO']
extrel.drop(['EXTRELNO'],axis=1,inplace=True)
extrel.head()

,EXTRELSTDT,Churned
EXTRELNO,,
0026419,1992-03-01,1
0026424,1992-03-01,0
0026430,1992-03-01,0
0026431,1992-03-01,1
0026443,1992-03-01,1


In [58]:
extrel = timestamp_column(extrel,'EXTRELSTDT')
extrel.head()

,Churned,EXTRELSTDT_year,EXTRELSTDT_month,EXTRELSTDT_day
EXTRELNO,,,,
0026419,1,1992,3,1
0026424,0,1992,3,1
0026430,0,1992,3,1
0026431,1,1992,3,1
0026443,1,1992,3,1


In [59]:
'''
Nameaddr: Sociodemographical information
Extrelno-Unique identifier of each donor
Name1title-Title to address someone
Postcode-Postcode
Languagecode-Preferred mailing language
'''
nameaddr=pd.read_sas("nameaddr.sas7bdat")

nameaddr['NAME1TITLE']=nameaddr.iloc[:,1].str.decode("utf-8")
nameaddr['EXTRELNO']=nameaddr.iloc[:,0].str.decode("utf-8")
nameaddr['POSTCODE']=nameaddr.iloc[:,2].str.decode("utf-8")
nameaddr['LANGUACODE']=nameaddr.iloc[:,3].str.decode("utf-8")

nameaddr.head()

,EXTRELNO,NAME1TITLE,POSTCODE,LANGUACODE
0,0000019,HR,2500,NL
1,0000020,HR,9550,NL
2,0000035,HM,2150,NL
3,0000042,FA,3012,NL
4,0000043,MW,3000,NL


In [60]:
nameaddr.sort_values(['EXTRELNO'],inplace=True)
nameaddr

,EXTRELNO,NAME1TITLE,POSTCODE,LANGUACODE
0,0000019,HR,2500,NL
1,0000020,HR,9550,NL
2,0000035,HM,2150,NL
3,0000042,FA,3012,NL
4,0000043,MW,3000,NL
5,0000047,HM,3360,NL
6,0000060,FA,3600,NL
7,0000072,MW,9000,NL
8,0000081,MW,8902,NL
9,0000089,FA,3600,NL


In [61]:
nameaddr.isnull().sum()

EXTRELNO        0
NAME1TITLE    593
POSTCODE      152
LANGUACODE      0
dtype: int64

In [62]:
nameaddr.drop_duplicates(['EXTRELNO'],inplace=True)

In [63]:
nameaddr.set_index('EXTRELNO',inplace=True)
nameaddr

,NAME1TITLE,POSTCODE,LANGUACODE
EXTRELNO,,,
0000019,HR,2500,NL
0000020,HR,9550,NL
0000035,HM,2150,NL
0000042,FA,3012,NL
0000043,MW,3000,NL
0000047,HM,3360,NL
0000060,FA,3600,NL
0000072,MW,9000,NL
0000081,MW,8902,NL


In [64]:
nameaddr.drop(['NAME1TITLE','POSTCODE'],axis=1,inplace=True)
nameaddr.head()

,LANGUACODE
EXTRELNO,
0000019,NL
0000020,NL
0000035,NL
0000042,NL
0000043,NL


In [65]:
nameaddr = pd.get_dummies(nameaddr,['LANGUACODE'])
nameaddr

,LANGUACODE_EN,LANGUACODE_FR,LANGUACODE_NL
EXTRELNO,,,
0000019,0,0,1
0000020,0,0,1
0000035,0,0,1
0000042,0,0,1
0000043,0,0,1
0000047,0,0,1
0000060,0,0,1
0000072,0,0,1
0000081,0,0,1


In [66]:
payhistory1.head()

,Frequency,TOTAL_DONATION,Average_donation,PAYTYPECD_D,PAYTYPECD_E,PAYTYPECD_O,PAYTYPECD_X,STATUS_CO,STATUS_OK,STATUS_RC,STATUS_RF,PDATE_year,PDATE_month,PDATE_day
EXTRELNO,,,,,,,,,,,,,,
0000019,1,9.57,9.570000,0,0,0,1,1,0,0,0,1993,12,31
0000020,158,3176.97,20.107405,1,0,0,1,0,1,0,0,2007,1,17
0000035,155,3168.40,20.441290,1,0,0,1,0,1,0,0,2007,1,17
0000042,85,1543.03,18.153294,1,0,0,1,0,0,1,0,2001,1,23
0000043,229,4373.24,19.097118,1,0,0,1,0,1,0,0,2003,7,14


In [67]:
communication.head()

,MEDIUMCODE_AC,MEDIUMCODE_AF,MEDIUMCODE_AM,MEDIUMCODE_CA,MEDIUMCODE_CI,MEDIUMCODE_EM,MEDIUMCODE_FF,MEDIUMCODE_FX,MEDIUMCODE_LE,MEDIUMCODE_ME,...,CLASCODE_ID,CLASCODE_NS,CLASCODE_PM,CLASCODE_RI,CLASCODE_RM,CONTDIREC_I,CONTDIREC_O,CONTDATE_year,CONTDATE_month,CONTDATE_day
EXTRELNO,,,,,,,,,,,,,,,,,,,,,
0000020,0,0,0,0,1,1,0,0,1,0,...,1,0,1,0,0,0,1,1994,5,9
0000035,0,0,0,0,1,1,0,0,1,0,...,1,0,1,0,0,1,1,2002,3,21
0000042,0,0,0,0,1,0,0,0,1,0,...,1,0,0,1,1,1,1,1998,12,3
0000043,0,0,0,0,1,1,0,0,1,0,...,1,0,1,1,0,1,1,2000,5,5
0000060,0,0,0,0,1,0,0,0,1,0,...,1,0,0,1,0,1,1,1998,3,20


In [68]:
extrel.head()

,Churned,EXTRELSTDT_year,EXTRELSTDT_month,EXTRELSTDT_day
EXTRELNO,,,,
0026419,1,1992,3,1
0026424,0,1992,3,1
0026430,0,1992,3,1
0026431,1,1992,3,1
0026443,1,1992,3,1


In [69]:
nameaddr.head()

,LANGUACODE_EN,LANGUACODE_FR,LANGUACODE_NL
EXTRELNO,,,
0000019,0,0,1
0000020,0,0,1
0000035,0,0,1
0000042,0,0,1
0000043,0,0,1


In [70]:
#Joining all the dataframes to form final data
data = pd.concat([payhistory1,communication,extrel,nameaddr],axis=1,join='inner')
data

,Frequency,TOTAL_DONATION,Average_donation,PAYTYPECD_D,PAYTYPECD_E,PAYTYPECD_O,PAYTYPECD_X,STATUS_CO,STATUS_OK,STATUS_RC,...,CONTDATE_year,CONTDATE_month,CONTDATE_day,Churned,EXTRELSTDT_year,EXTRELSTDT_month,EXTRELSTDT_day,LANGUACODE_EN,LANGUACODE_FR,LANGUACODE_NL
EXTRELNO,,,,,,,,,,,,,,,,,,,,,
0026419,69,3155.05,45.725362,1,1,1,1,0,1,0,...,2002,11,8,1,1992,3,1,0,0,1
0026424,35,524.85,14.995714,0,0,1,1,0,1,0,...,1996,4,1,0,1992,3,1,0,0,1
0026430,146,2786.13,19.083082,1,0,1,1,0,1,0,...,2002,2,16,0,1992,3,1,0,0,1
0026431,21,3050.61,145.267143,1,1,1,1,0,1,0,...,2003,4,11,1,1992,3,1,0,0,1
0026443,156,3109.62,19.933462,1,0,0,1,0,1,0,...,2003,4,11,1,1992,3,1,0,0,1
0026444,157,3092.47,19.697261,1,0,0,1,0,1,0,...,2002,7,12,1,1992,3,1,0,0,1
0026446,66,3667.54,55.568788,1,0,0,1,0,1,0,...,2000,2,12,1,1992,3,1,0,0,1
0026447,156,3110.13,19.936731,1,0,0,1,0,1,0,...,2002,11,8,1,1992,3,1,0,0,1
0026462,31,1602.10,51.680645,0,0,1,1,0,1,0,...,1998,5,5,0,1992,3,1,0,1,0


In [71]:
data.isnull().sum().any()

False

In [72]:
data.columns

Index(['Frequency', 'TOTAL_DONATION', 'Average_donation', 'PAYTYPECD_D',
       'PAYTYPECD_E', 'PAYTYPECD_O', 'PAYTYPECD_X', 'STATUS_CO', 'STATUS_OK',
       'STATUS_RC', 'STATUS_RF', 'PDATE_year', 'PDATE_month', 'PDATE_day',
       'MEDIUMCODE_AC', 'MEDIUMCODE_AF', 'MEDIUMCODE_AM', 'MEDIUMCODE_CA',
       'MEDIUMCODE_CI', 'MEDIUMCODE_EM', 'MEDIUMCODE_FF', 'MEDIUMCODE_FX',
       'MEDIUMCODE_LE', 'MEDIUMCODE_ME', 'MEDIUMCODE_MR', 'MEDIUMCODE_NA',
       'MEDIUMCODE_RM', 'MEDIUMCODE_TE', 'MEDIUMCODE_TM', 'MNTOPCODE_ALL',
       'MNTOPCODE_ARR', 'MNTOPCODE_CAN', 'MNTOPCODE_CAT', 'MNTOPCODE_COR',
       'MNTOPCODE_CTS', 'MNTOPCODE_DIS', 'MNTOPCODE_ENR', 'MNTOPCODE_FIC',
       'MNTOPCODE_FIN', 'MNTOPCODE_IFF', 'MNTOPCODE_INF', 'MNTOPCODE_IRE',
       'MNTOPCODE_MER', 'MNTOPCODE_PRA', 'MNTOPCODE_PRO', 'MNTOPCODE_REP',
       'MNTOPCODE_SPO', 'MNTOPCODE_TAX', 'MNTOPCODE_TLT', 'MNTOPCODE_TVS',
       'MNTOPCODE_VIS', 'MNTOPCODE_VOL', 'MNTOPCODE_WEB', 'CLASCODE_AK',
       'CLASCODE_AN', 'CLA

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10780 entries, 0026419 to 0244875
Data columns (total 74 columns):
Frequency           10780 non-null int64
TOTAL_DONATION      10780 non-null float64
Average_donation    10780 non-null float64
PAYTYPECD_D         10780 non-null int64
PAYTYPECD_E         10780 non-null int64
PAYTYPECD_O         10780 non-null int64
PAYTYPECD_X         10780 non-null int64
STATUS_CO           10780 non-null uint8
STATUS_OK           10780 non-null uint8
STATUS_RC           10780 non-null uint8
STATUS_RF           10780 non-null uint8
PDATE_year          10780 non-null int64
PDATE_month         10780 non-null int64
PDATE_day           10780 non-null int64
MEDIUMCODE_AC       10780 non-null int64
MEDIUMCODE_AF       10780 non-null int64
MEDIUMCODE_AM       10780 non-null int64
MEDIUMCODE_CA       10780 non-null int64
MEDIUMCODE_CI       10780 non-null int64
MEDIUMCODE_EM       10780 non-null int64
MEDIUMCODE_FF       10780 non-null int64
MEDIUMCODE_FX       107

# Removing Outliers 

In [75]:
def var_summary(x):
   return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  x.std(), x.var(), x.min(), x.dropna().quantile(0.01), x.dropna().quantile(0.05),x.dropna().quantile(0.10),x.dropna().quantile(0.25),x.dropna().quantile(0.50),x.dropna().quantile(0.75), x.dropna().quantile(0.90),x.dropna().quantile(0.95), x.dropna().quantile(0.99),x.max()], 
                 index=['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1' , 'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])

num_summary=data.apply(lambda x: var_summary(x)).T
num_summary

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
Frequency,10780.0,0.0,7.357810e+05,68.254267,53.000000,59.001278,3.481151e+03,1.0,1.000000,4.000000,8.900000,21.00,53.000000,109.000000,153.000000,156.0,216.2100,1510.00
TOTAL_DONATION,10780.0,0.0,1.942589e+07,1802.030240,1768.575000,1253.348160,1.570882e+06,-875.0,7.455000,121.980000,276.936000,777.03,1768.575000,2925.902500,3111.290000,3129.6,5334.6971,30860.49
Average_donation,10780.0,0.0,4.485486e+05,41.609329,22.320235,48.644528,2.366290e+03,-138.0,2.016467,15.716752,18.426058,20.33,22.320235,39.994286,92.259655,149.5,263.9775,600.00
PAYTYPECD_D,10780.0,0.0,8.354000e+03,0.774954,1.000000,0.417632,1.744167e-01,0.0,0.000000,0.000000,0.000000,1.00,1.000000,1.000000,1.000000,1.0,1.0000,1.00
PAYTYPECD_E,10780.0,0.0,2.920000e+03,0.270872,0.000000,0.444431,1.975187e-01,0.0,0.000000,0.000000,0.000000,0.00,0.000000,1.000000,1.000000,1.0,1.0000,1.00
PAYTYPECD_O,10780.0,0.0,6.230000e+03,0.577922,1.000000,0.493914,2.439508e-01,0.0,0.000000,0.000000,0.000000,0.00,1.000000,1.000000,1.000000,1.0,1.0000,1.00
PAYTYPECD_X,10780.0,0.0,2.600000e+03,0.241187,0.000000,0.427824,1.830330e-01,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,1.0,1.0000,1.00
STATUS_CO,10780.0,0.0,3.790000e+02,0.035158,0.000000,0.184187,3.392478e-02,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,1.0000,1.00
STATUS_OK,10780.0,0.0,9.756000e+03,0.905009,1.000000,0.293216,8.597546e-02,0.0,0.000000,0.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.0,1.0000,1.00
STATUS_RC,10780.0,0.0,6.280000e+02,0.058256,0.000000,0.234238,5.486735e-02,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,1.0,1.0000,1.00


In [76]:
# outlier columns are the name of the columns having outliers 
# since we can't remove outliers from out target variable,that is why i'am removing "Total_Cardspent" from the list.

outlier_columns = list(num_summary[num_summary['MAX']>num_summary['P99']].index)
outlier_columns

['Frequency',
 'TOTAL_DONATION',
 'Average_donation',
 'STATUS_RF',
 'PDATE_day',
 'MEDIUMCODE_AC',
 'MEDIUMCODE_AM',
 'MEDIUMCODE_TM',
 'MNTOPCODE_SPO',
 'MNTOPCODE_TLT',
 'MNTOPCODE_WEB',
 'CLASCODE_NS',
 'CONTDATE_day',
 'EXTRELSTDT_year']

In [78]:
# For Removing Outliers 
for x in outlier_columns:
    data[x]=data[x].clip_upper(data[x].quantile(0.99))
    data[x]=data[x].clip_lower(data[x].quantile(0.01))

In [80]:
removed_outliers = data.apply(lambda x: var_summary(x)).T
removed_outliers[removed_outliers['MAX']>removed_outliers['P99']]

# now You can see that the outliers are removed upto some extent

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
Frequency,10780.0,0.0,7.254347e+05,67.294497,53.000,53.464019,2.858401e+03,1.000,1.00000,4.00,8.900,21.00,53.000,109.0000,153.00,156.0,216.044100,216.2100
TOTAL_DONATION,10780.0,0.0,1.923066e+07,1783.920439,1768.575,1132.782288,1.283196e+06,7.455,7.53795,121.98,276.936,777.03,1768.575,2925.9025,3111.29,3129.6,5333.119391,5334.6971
CONTDATE_day,10780.0,0.0,1.623267e+05,15.058134,15.000,8.554170,7.317383e+01,1.000,1.00000,1.00,3.000,8.00,15.000,22.0000,27.00,29.0,30.044100,30.2100


In [81]:
X = data.drop(['Churned'],axis=1) # Independent Columns
y = data['Churned']       # Dependent Columns

In [82]:
X.head()

,Frequency,TOTAL_DONATION,Average_donation,PAYTYPECD_D,PAYTYPECD_E,PAYTYPECD_O,PAYTYPECD_X,STATUS_CO,STATUS_OK,STATUS_RC,...,CONTDIREC_O,CONTDATE_year,CONTDATE_month,CONTDATE_day,EXTRELSTDT_year,EXTRELSTDT_month,EXTRELSTDT_day,LANGUACODE_EN,LANGUACODE_FR,LANGUACODE_NL
EXTRELNO,,,,,,,,,,,,,,,,,,,,,
0026419,69.0,3155.05,45.725362,1,1,1,1,0,1,0,...,1,2002,11,8.0,1992.0,3,1,0,0,1
0026424,35.0,524.85,14.995714,0,0,1,1,0,1,0,...,1,1996,4,1.0,1992.0,3,1,0,0,1
0026430,146.0,2786.13,19.083082,1,0,1,1,0,1,0,...,1,2002,2,16.0,1992.0,3,1,0,0,1
0026431,21.0,3050.61,145.267143,1,1,1,1,0,1,0,...,1,2003,4,11.0,1992.0,3,1,0,0,1
0026443,156.0,3109.62,19.933462,1,0,0,1,0,1,0,...,1,2003,4,11.0,1992.0,3,1,0,0,1


In [83]:
y.head()

EXTRELNO
0026419    1
0026424    0
0026430    0
0026431    1
0026443    1
Name: Churned, dtype: int64

In [84]:
from sklearn.model_selection import KFold 

kf = KFold(n_splits=5,shuffle=True,random_state=None)
for train_index, test_index in kf.split(X):
    print("Train:", train_index, "Validation:",test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index] 
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

Train: [    0     3     4 ... 10776 10777 10778] Validation: [    1     2     6 ... 10769 10773 10779]
Train: [    0     1     2 ... 10777 10778 10779] Validation: [    3    10    19 ... 10754 10757 10761]
Train: [    0     1     2 ... 10777 10778 10779] Validation: [    8     9    12 ... 10772 10774 10776]
Train: [    1     2     3 ... 10776 10778 10779] Validation: [    0     4    14 ... 10771 10775 10777]
Train: [    0     1     2 ... 10776 10777 10779] Validation: [    5    13    20 ... 10764 10766 10778]


In [85]:
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [86]:
y_pred_train = rfc.predict(X_train)

print("The accuracy score came out to be on train : {}".format(metrics.accuracy_score(y_train,y_pred_train)))
print("The precision score came out to be on train : {}".format(metrics.precision_score(y_train,y_pred_train)))
print("The recall score came out to be on train : {}".format(metrics.recall_score(y_train,y_pred_train)))

The accuracy score came out to be on train : 0.99930426716141
The precision score came out to be on train : 0.9994606256742179
The recall score came out to be on train : 0.9994606256742179


In [87]:
y_pred_test = rfc.predict(X_test)

print("The accuracy score came out to be on test : {}".format(metrics.accuracy_score(y_test,y_pred_test)))
print("The precision score came out to be on test : {}".format(metrics.precision_score(y_test,y_pred_test)))
print("The recall score came out to be on test : {}".format(metrics.recall_score(y_test,y_pred_test)))

The accuracy score came out to be on test : 0.9897959183673469
The precision score came out to be on test : 0.9927060539752006
The recall score came out to be on test : 0.9912600145666424


In [88]:
metrics.confusion_matrix(y_test,y_pred_test)

array([[ 773,   10],
       [  12, 1361]], dtype=int64)

In [89]:
pd.DataFrame({'Actual':y_test ,'Predicted':y_pred_test})

,Actual,Predicted
EXTRELNO,,
0026444,1,1
0026481,0,0
0026506,0,0
0026560,1,1
0026578,0,0
0026655,1,1
0026686,0,0
0026693,1,1
0026706,1,1


# Feature Selection 

In [90]:
rfc.feature_importances_

array([2.65488158e-02, 3.98097630e-02, 4.31176718e-02, 1.35255720e-03,
       1.13443441e-02, 1.02568889e-03, 7.61580869e-04, 1.04466962e-03,
       2.32288507e-02, 1.49888939e-02, 0.00000000e+00, 1.81628629e-01,
       3.67037003e-02, 2.16727314e-02, 0.00000000e+00, 1.91458588e-04,
       0.00000000e+00, 1.12994650e-04, 1.76390529e-05, 8.81555924e-02,
       1.08031017e-03, 4.41798294e-04, 0.00000000e+00, 7.06391377e-04,
       6.43980178e-04, 0.00000000e+00, 7.52073706e-03, 1.55349212e-03,
       0.00000000e+00, 4.43180397e-04, 1.94653855e-02, 1.12504773e-03,
       1.09972350e-04, 3.77987282e-03, 8.93800819e-05, 2.69206922e-01,
       1.08411996e-03, 5.24078859e-04, 1.16970195e-03, 2.78100773e-04,
       1.14623428e-03, 9.27638346e-04, 4.10927807e-04, 4.26349226e-04,
       9.18415187e-04, 1.48040530e-03, 0.00000000e+00, 2.76793900e-04,
       0.00000000e+00, 2.32489721e-04, 3.36506784e-04, 4.74300021e-04,
       0.00000000e+00, 5.94323367e-03, 2.86079439e-03, 2.52889325e-04,
      

In [91]:
Feature_selection = pd.DataFrame({"Features":X_train.columns,"Importance":rfc.feature_importances_})


selected = list(Feature_selection[Feature_selection['Importance']>0.005].Features)
selected

['Frequency',
 'TOTAL_DONATION',
 'Average_donation',
 'PAYTYPECD_E',
 'STATUS_OK',
 'STATUS_RC',
 'PDATE_year',
 'PDATE_month',
 'PDATE_day',
 'MEDIUMCODE_EM',
 'MEDIUMCODE_RM',
 'MNTOPCODE_ARR',
 'MNTOPCODE_DIS',
 'CLASCODE_AK',
 'CLASCODE_PM',
 'CLASCODE_RM',
 'CONTDIREC_I',
 'CONTDATE_year',
 'EXTRELSTDT_year']

In [92]:
# Updating the independent variables 
X = X.loc[:,selected]

In [93]:
# Again splitting data into Train and Test

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y)

In [94]:
rfc1 = RandomForestClassifier()
rfc1.fit(X_train,y_train)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [95]:
y_pred_train = rfc1.predict(X_train)

print("The accuracy score came out to be on train : {}".format(metrics.accuracy_score(y_train,y_pred_train)))
print("The precision score came out to be on train : {}".format(metrics.precision_score(y_train,y_pred_train)))
print("The recall score came out to be on train : {}".format(metrics.recall_score(y_train,y_pred_train)))

The accuracy score came out to be on train : 0.9988868274582561
The precision score came out to be on train : 0.9996142719382836
The recall score came out to be on train : 0.9986512524084779


In [96]:
y_pred_test = rfc1.predict(X_test)

print("The accuracy score came out to be on test : {}".format(metrics.accuracy_score(y_test,y_pred_test)))
print("The precision score came out to be on test : {}".format(metrics.precision_score(y_test,y_pred_test)))
print("The recall score came out to be on test : {}".format(metrics.recall_score(y_test,y_pred_test)))

The accuracy score came out to be on test : 0.9925788497217068
The precision score came out to be on test : 0.9971181556195965
The recall score came out to be on test : 0.9914040114613181


In [97]:
metrics.confusion_matrix(y_test,y_pred_test)

array([[ 945,    5],
       [  15, 1730]], dtype=int64)

In [98]:
pd.DataFrame({'Actual':y_test ,'Predicted':y_pred_test})

,Actual,Predicted
EXTRELNO,,
0233226,1,1
0211472,0,0
0217900,1,1
0026948,1,1
0215332,0,0
0198003,1,1
0174824,0,0
0037969,1,1
0225477,1,1
